In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

# Define dataset path
data_dir = "./plant_dataset"  # Change this to your dataset path

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# Data Augmentation & Preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    subset='validation'
)

# Define class names
class_names = list(train_generator.class_indices.keys())

# Load MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model

# Define model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(class_names), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=8, validation_data=val_generator)

# Evaluate the model
loss, accuracy = model.evaluate(val_generator)
print(f'Test Accuracy: {accuracy:.2f}')

# Save the model
model.save("plant_disease_model.h5")


Found 49197 images belonging to 39 classes.
Found 12289 images belonging to 39 classes.
Epoch 1/15
1538/1538 [==============================] - 2244s 1s/step - loss: 0.8445 - accuracy: 0.7478 - val_loss: 0.3711 - val_accuracy: 0.8818
Epoch 2/15
1538/1538 [==============================] - 658s 428ms/step - loss: 0.4810 - accuracy: 0.8457 - val_loss: 0.2957 - val_accuracy: 0.9042
Epoch 3/15
1538/1538 [==============================] - 664s 432ms/step - loss: 0.4421 - accuracy: 0.8578 - val_loss: 0.2798 - val_accuracy: 0.9076
Epoch 4/15
1538/1538 [==============================] - 724s 471ms/step - loss: 0.4100 - accuracy: 0.8683 - val_loss: 0.2596 - val_accuracy: 0.9120
Epoch 5/15
1538/1538 [==============================] - 735s 478ms/step - loss: 0.4012 - accuracy: 0.8706 - val_loss: 0.2494 - val_accuracy: 0.9177
Epoch 6/15
1538/1538 [==============================] - 663s 431ms/step - loss: 0.3748 - accuracy: 0.8777 - val_loss: 0.2332 - val_accuracy: 0.9221
Epoch 7/15
1538/1538 [====